In [1]:
import sys
sys.path.append("..")


In [3]:
import sys
print(sys.executable)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\python.exe


In [4]:
C:\path\to\python.exe -m pip install truststore

SyntaxError: unexpected character after line continuation character (3712867574.py, line 1)

In [6]:
import truststore
print(truststore.__version__)


0.10.4


In [7]:
import time

import httpx
import pandas as pd
import xml.etree.ElementTree as ET

from utils import ssl_context, httpx_req_headers, retry_decorator

In [8]:
import sys
print(sys.executable)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\python.exe


In [9]:
def sitemap_to_list(response: httpx.Response):
    # response = requests.get(url)
    root = ET.fromstring(response.content)

    ns = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    sitemap_data = []

    url_elems = root.findall('ns:url', ns)
    sitemap_elems = root.findall('ns:sitemap', ns)
    if url_elems:
        findall_list = url_elems
    elif sitemap_elems:
        findall_list = sitemap_elems
    else:
        raise ValueError(f"No url and sitemap found in {response.content}")


    for url_elem in findall_list:
        entry = {}
        loc = url_elem.find('ns:loc', ns)
        lastmod = url_elem.find('ns:lastmod', ns)
        changefreq = url_elem.find('ns:changefreq', ns)
        priority = url_elem.find('ns:priority', ns)

        if loc is not None:
            entry['loc'] = loc.text
        if lastmod is not None:
            entry['lastmod'] = lastmod.text
        if changefreq is not None:
            entry['changefreq'] = changefreq.text
        if priority is not None:
            entry['priority'] = priority.text

        sitemap_data.append(entry)
    print(len(sitemap_data))

    return sitemap_data

In [10]:
@retry_decorator(retries=3, delay=10)
def req_support_function(client: httpx.Client, url:str) -> httpx.Response:

    res = client.get(url)

    elem_list = sitemap_to_list(res)

    return elem_list
    

In [15]:
with httpx.Client(http2=True, follow_redirects=True, verify=ssl_context, headers=httpx_req_headers, timeout=30) as client:

    sitemap_index = "https://channelstore.roku.com/sitemap-index.xml"
    roku_stores = req_support_function(client, sitemap_index)

roku_stores

'<?xml version="1.0" encoding="UTF-8"?><sitemapindex xmlns="http://www.sitemaps.org/schemas/sitemap/0.9"><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/en-ot/sitemap.xml</loc>\n                    </sitemap><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/en-us/sitemap.xml</loc>\n                    </sitemap><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/es-us/sitemap.xml</loc>\n                    </sitemap><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/en-ca/sitemap.xml</loc>\n                    </sitemap><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/fr-ca/sitemap.xml</loc>\n                    </sitemap><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/en-gb/sitemap.xml</loc>\n                    </sitemap><sitemap>\n                        <loc>https://channelstore.roku.com/sitemaps/fr-fr/sitema

In [22]:
import httpx
import ssl
import xml.etree.ElementTree as ET

ssl_context = ssl.create_default_context()
httpx_req_headers = {"User-Agent": "Mozilla/5.0"}

def req_support_function(client, url):
    response = client.get(url)
    response.raise_for_status()
    return response.text

with httpx.Client(http2=True, follow_redirects=True, verify=ssl_context, headers=httpx_req_headers, timeout=30) as client:
    sitemap_index = "https://channelstore.roku.com/sitemap-index.xml"
    xml_content = req_support_function(client, sitemap_index)
    
    # Parse XML with namespace
    root = ET.fromstring(xml_content)
    ns = {'s': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    
    # Extract all <loc> elements
    roku_stores = [elem.text for elem in root.findall(".//s:loc", ns)]

# Now roku_stores is ready
print(roku_stores[0])


https://channelstore.roku.com/sitemaps/en-ot/sitemap.xml


In [25]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import httpx
import ssl

ssl_context = ssl.create_default_context()
httpx_req_headers = {"User-Agent": "Mozilla/5.0"}

def req_support_function(client, url):
    response = client.get(url)
    response.raise_for_status()
    return response.text  # raw XML

full_df_list = []

for idx, sitemap_url in enumerate(roku_stores, start=1):
    print(f"{idx} - running the sitemap: {sitemap_url}")
    
    with httpx.Client(http2=True, follow_redirects=True, verify=ssl_context, headers=httpx_req_headers, timeout=90) as client:
        xml_content = req_support_function(client, sitemap_url)
        
        # Parse XML for <loc> URLs (apps)
        root = ET.fromstring(xml_content)
        ns = {'s': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        
        app_urls = [{'loc': elem.text} for elem in root.findall(".//s:loc", ns)]
        
        # Convert to DataFrame
        df = pd.DataFrame(app_urls)
        df["sitemap_url"] = sitemap_url
        
        full_df_list.append(df)
        
        time.sleep(30)  # polite delay


1 - running the sitemap: https://channelstore.roku.com/sitemaps/en-ot/sitemap.xml
2 - running the sitemap: https://channelstore.roku.com/sitemaps/en-us/sitemap.xml
3 - running the sitemap: https://channelstore.roku.com/sitemaps/es-us/sitemap.xml
4 - running the sitemap: https://channelstore.roku.com/sitemaps/en-ca/sitemap.xml
5 - running the sitemap: https://channelstore.roku.com/sitemaps/fr-ca/sitemap.xml
6 - running the sitemap: https://channelstore.roku.com/sitemaps/en-gb/sitemap.xml
7 - running the sitemap: https://channelstore.roku.com/sitemaps/fr-fr/sitemap.xml
8 - running the sitemap: https://channelstore.roku.com/sitemaps/es-mx/sitemap.xml
9 - running the sitemap: https://channelstore.roku.com/sitemaps/es-co/sitemap.xml
10 - running the sitemap: https://channelstore.roku.com/sitemaps/es-ar/sitemap.xml
11 - running the sitemap: https://channelstore.roku.com/sitemaps/es-pe/sitemap.xml
12 - running the sitemap: https://channelstore.roku.com/sitemaps/es-cl/sitemap.xml
13 - running 

In [26]:
full_df = pd.concat(full_df_list)
full_df

,loc,sitemap_url
0,https://channelstore.roku.com/details/6b51d431...,https://channelstore.roku.com/sitemaps/en-ot/s...
1,https://channelstore.roku.com/details/8527a891...,https://channelstore.roku.com/sitemaps/en-ot/s...
2,https://channelstore.roku.com/details/501a4e61...,https://channelstore.roku.com/sitemaps/en-ot/s...
3,https://channelstore.roku.com/details/53d2dd25...,https://channelstore.roku.com/sitemaps/en-ot/s...
4,https://channelstore.roku.com/details/63fd44ee...,https://channelstore.roku.com/sitemaps/en-ot/s...
...,...,...
13044,https://channelstore.roku.com/details/1060420b...,https://channelstore.roku.com/sitemaps/en-au/s...
13045,https://channelstore.roku.com/details/74d5d634...,https://channelstore.roku.com/sitemaps/en-au/s...
13046,https://channelstore.roku.com/details/bcd2504d...,https://channelstore.roku.com/sitemaps/en-au/s...
13047,https://channelstore.roku.com/details/d5f3d2fd...,https://channelstore.roku.com/sitemaps/en-au/s...


In [27]:
full_df.to_csv("NEW_roku_store_apps.csv", index=False)

In [28]:
full_df['loc'].unique()

array(['https://channelstore.roku.com/details/6b51d431df5d7f141cbececcf79edf3d:f4f106825aa59985b092fc699f825ca0/netflix',
       'https://channelstore.roku.com/details/8527a891e224136950ff32ca212b45bc:efdff7389cbb4a5d24c7fee438b4788f/mlb',
       'https://channelstore.roku.com/details/501a4e61aa4f7737df0305124a39119b:9f0f785535ad319cd05f98e8af0171c6/youtube',
       ...,
       'https://channelstore.roku.com/details/e38ef3947369d388414b004e6eed52c6:325c14c132b352c99dee7704ff023bf0/veterans-outreach-ministries',
       'https://channelstore.roku.com/details/3cd754dd3d5b57e92a379b6a469fc359:a2e27e897576b55b0be45f103254bf4e/kayosports',
       'https://channelstore.roku.com/details/2596daca6adc757ac332f0f776264f0e:bf5eafd24bd0ad001b4ea4716f45fac1/binge'],
      shape=(77035,), dtype=object)

In [8]:
from lxml import html
from typing import Dict, Optional

def extract_appstore_meta_tags(content) -> Dict[str, Optional[str]]:
    # with open(html_path, "r", encoding="utf-8") as f:
    #     content = f.read()

    tree = html.fromstring(content)

    def get_meta_content(name: str) -> Optional[str]:
        result = tree.xpath(f'//meta[@name="{name}"]/@content')
        return result[0] if result else None

    return {
        "appstore:store_id": get_meta_content("appstore:store_id"),
        "appstore:bundle_id": get_meta_content("appstore:bundle_id"),
        "appstore:developer_url": get_meta_content("appstore:developer_url")
    }


In [9]:
import curl_cffi

with curl_cffi.Session() as client:
    res = client.get("https://channelstore.roku.com/details/74e07738778cf9dfb74340ef94503257:f9694b0de646eb20d6f3f14e2618994b/railstream", impersonate="chrome", allow_redirects=True)
    print(res.status_code)

200


In [22]:
extract_appstore_meta_tags(res.text)

{'appstore:store_id': '182257',
 'appstore:bundle_id': '182257',
 'appstore:developer_url': 'https://railstream.net'}

In [6]:
import pandas as pd
df = pd.read_csv("roku_store_apps.csv")
df['loc'].unique()

array(['https://channelstore.roku.com/details/6b51d431df5d7f141cbececcf79edf3d:f4f106825aa59985b092fc699f825ca0/netflix',
       'https://channelstore.roku.com/details/8527a891e224136950ff32ca212b45bc:efdff7389cbb4a5d24c7fee438b4788f/mlb',
       'https://channelstore.roku.com/details/501a4e61aa4f7737df0305124a39119b:9f0f785535ad319cd05f98e8af0171c6/youtube',
       ...,
       'https://channelstore.roku.com/details/0e99423e685eba5d3b6a00efc103b924:9019f2ef4241befdb4b1a0990f8c9504/vegas-beyond-the-strip',
       'https://channelstore.roku.com/details/11683abfb2b6a3f468da680119df78d1:8e4febcda81a739ef9cc9c0ea09d0b29/miami-beach-fm',
       'https://channelstore.roku.com/details/20a1431ce07b79d8384ab6ddba48c7bc:242de2e1052e1e8d8bb7d7da5f2cb7f8/tank-shooter'],
      shape=(68525,), dtype=object)

In [ ]:
import time
import curl_cffi

from utils import retry_decorator

roku_app_data = []
error_data = []

@retry_decorator(delay=20)
def helper_req(client: curl_cffi.Session, url):

    res = client.get(url, impersonate="chrome", allow_redirects=True, timeout=15)
    res.raise_for_status()

    return res

with curl_cffi.Session() as client:
    for idx, url in enumerate(df['loc'].unique(), start=1):
        print(f"{idx} - {url=}")
        try:
            res = helper_req(client, url)
            res_status_code = res.status_code
            print(res_status_code)
            meta_data = extract_appstore_meta_tags(res.text)
            meta_data['url'] = url
            meta_data['status_code'] = res_status_code
            print(meta_data)
            roku_app_data.append(meta_data)
        except Exception as e:
            print(f"Exception for the {url=} \n{e}")

        time.sleep(15)



1 - url='https://channelstore.roku.com/details/6b51d431df5d7f141cbececcf79edf3d:f4f106825aa59985b092fc699f825ca0/netflix'
Running (1): helper_req()
200
{'appstore:store_id': '12', 'appstore:bundle_id': '12', 'appstore:developer_url': None, 'url': 'https://channelstore.roku.com/details/6b51d431df5d7f141cbececcf79edf3d:f4f106825aa59985b092fc699f825ca0/netflix', 'status_code': 200}
2 - url='https://channelstore.roku.com/details/8527a891e224136950ff32ca212b45bc:efdff7389cbb4a5d24c7fee438b4788f/mlb'
Running (1): helper_req()
200
{'appstore:store_id': '14', 'appstore:bundle_id': '14', 'appstore:developer_url': 'https://www.mlb.com', 'url': 'https://channelstore.roku.com/details/8527a891e224136950ff32ca212b45bc:efdff7389cbb4a5d24c7fee438b4788f/mlb', 'status_code': 200}
3 - url='https://channelstore.roku.com/details/501a4e61aa4f7737df0305124a39119b:9f0f785535ad319cd05f98e8af0171c6/youtube'
Running (1): helper_req()
200
{'appstore:store_id': '837', 'appstore:bundle_id': '837', 'appstore:develop

KeyboardInterrupt: 

In [14]:
roku_app_data

[{'appstore:store_id': '12',
  'appstore:bundle_id': '12',
  'appstore:developer_url': None,
  'url': 'https://channelstore.roku.com/details/6b51d431df5d7f141cbececcf79edf3d:f4f106825aa59985b092fc699f825ca0/netflix',
  'status_code': 200},
 {'appstore:store_id': '14',
  'appstore:bundle_id': '14',
  'appstore:developer_url': 'https://www.mlb.com',
  'url': 'https://channelstore.roku.com/details/8527a891e224136950ff32ca212b45bc:efdff7389cbb4a5d24c7fee438b4788f/mlb',
  'status_code': 200},
 {'appstore:store_id': '837',
  'appstore:bundle_id': '837',
  'appstore:developer_url': None,
  'url': 'https://channelstore.roku.com/details/501a4e61aa4f7737df0305124a39119b:9f0f785535ad319cd05f98e8af0171c6/youtube',
  'status_code': 200},
 {'appstore:store_id': '1453',
  'appstore:bundle_id': '1453',
  'appstore:developer_url': None,
  'url': 'https://channelstore.roku.com/details/53d2dd2504402eec1bc49ad74daf2e90:a13d7afc25f9a0a2db48abf783f2a46e/tunein',
  'status_code': 200},
 {'appstore:store_id':

In [16]:
roku_df_one = pd.DataFrame(roku_app_data)
roku_df_one

,appstore:store_id,appstore:bundle_id,appstore:developer_url,url,status_code
0,12,12,None,https://channelstore.roku.com/details/6b51d431...,200
1,14,14,https://www.mlb.com,https://channelstore.roku.com/details/8527a891...,200
2,837,837,None,https://channelstore.roku.com/details/501a4e61...,200
3,1453,1453,None,https://channelstore.roku.com/details/53d2dd25...,200
4,1457,1457,None,https://channelstore.roku.com/details/63fd44ee...,200
...,...,...,...,...,...
157,None,None,None,https://channelstore.roku.com/details/0d7a4916...,200
158,21569,21569,None,https://channelstore.roku.com/details/130fb904...,200
159,21702,21702,None,https://channelstore.roku.com/details/5be2dab0...,200
160,21727,21727,None,https://channelstore.roku.com/details/970ceca0...,200


In [17]:
roku_df_one.to_csv("roku_meta_data_1.csv")

In [ ]:
# import asyncio
# from curl_cffi.requests import AsyncSession
# from asyncio import Semaphore
# import json
# import os

# semaphore = Semaphore(5)
# urls = df['loc'].unique()

# # Ensure output directory exists
# os.makedirs("responses", exist_ok=True)

# async def fetch_with_retry(session: AsyncSession, url: str, retries: int = 3, timeout: int = 5):
#     attempt = 0
#     while attempt < retries:
#         async with semaphore:
#             try:
#                 response = await session.get(url, timeout=timeout)
#                 print(f"[{response.status_code}] {url}")

#                 data = response.json()

#                 # Save to file
#                 filename = f"responses/response_{url.split('=')[-1]}.json"
#                 with open(filename, "w") as f:
#                     json.dump(data, f, indent=2)

#                 return  # Exit on success

#             except Exception as e:
#                 print(f"Retry {attempt+1} failed for {url}: {e}")
#                 attempt += 1
#                 await asyncio.sleep(1)

# async def main():
#     async with AsyncSession() as session:
#         tasks = [fetch_with_retry(session, url) for url in urls]
#         await asyncio.gather(*tasks)

# if __name__ == "__main__":
#     asyncio.run(main())


In [3]:
import pandas as pd

df = pd.read_csv("../output_data/roku_store_apps.csv")
df_already_done = pd.read_parquet(r"C:\Users\zaid\Downloads\roku_app_data.parquet").drop_duplicates(inplace=False)

In [4]:
df_already_done

,appstore_store_id,appstore_bundle_id,appstore_developer_url,url,status_code
0,14,14,https://www.mlb.com,https://channelstore.roku.com/details/8527a891...,200
1,12,12,,https://channelstore.roku.com/details/6b51d431...,200
2,1457,1457,,https://channelstore.roku.com/details/63fd44ee...,200
3,1453,1453,,https://channelstore.roku.com/details/53d2dd25...,200
4,837,837,,https://channelstore.roku.com/details/501a4e61...,200
...,...,...,...,...,...
95,14273,14273,,https://channelstore.roku.com/details/214ed1d8...,200
96,14422,14422,,https://channelstore.roku.com/details/390490d4...,200
97,14487,14487,,https://channelstore.roku.com/details/fdd97d72...,200
98,14686,14686,,https://channelstore.roku.com/details/0c3dd06f...,200


In [5]:
urls = df['loc'].unique()
urls

array(['https://channelstore.roku.com/details/6b51d431df5d7f141cbececcf79edf3d:f4f106825aa59985b092fc699f825ca0/netflix',
       'https://channelstore.roku.com/details/8527a891e224136950ff32ca212b45bc:efdff7389cbb4a5d24c7fee438b4788f/mlb',
       'https://channelstore.roku.com/details/501a4e61aa4f7737df0305124a39119b:9f0f785535ad319cd05f98e8af0171c6/youtube',
       ...,
       'https://channelstore.roku.com/details/0e99423e685eba5d3b6a00efc103b924:9019f2ef4241befdb4b1a0990f8c9504/vegas-beyond-the-strip',
       'https://channelstore.roku.com/details/11683abfb2b6a3f468da680119df78d1:8e4febcda81a739ef9cc9c0ea09d0b29/miami-beach-fm',
       'https://channelstore.roku.com/details/20a1431ce07b79d8384ab6ddba48c7bc:242de2e1052e1e8d8bb7d7da5f2cb7f8/tank-shooter'],
      shape=(68525,), dtype=object)